In [ ]:
import av
import torch
import numpy as np
from transformers import SiglipImageProcessor
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from transformers import SiglipModel
from tqdm import tqdm

import json

DEVICE= torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len, num_segment, chunk_nb):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    
    end_idx = seg_len - 1
    start_idx = 0
    indices = np.linspace(start_idx, end_idx, num=num_segment * clip_len)[chunk_nb*clip_len:(chunk_nb+1)*clip_len]
    indices = np.clip(indices, 0, seg_len - 1).astype(np.int64)
    return indices

In [ ]:
class VideoClsDataset(Dataset):
    def __init__(self, anno_path, prefix, processor, num_segment) -> None:
        super().__init__()
        cleaned = pd.read_csv(anno_path, header=None, delimiter=' ')
        self.samples = list(cleaned.values[:, 0])
        self.labels = list(cleaned.values[:, 1])
        self.num_segment = num_segment
        
        self.label_array = list()
        self.dataset_samples = list()
        for i in range(self.num_segment):
            for sample, label in zip(self.samples, self.labels):
                self.label_array.append((i, label))
                self.dataset_samples.append(sample)
            
        self.prefix = prefix
        self.processor = processor
    def __len__(self):
        return len(self.label_array)
    def __getitem__(self, index):
        video_file = os.path.join(self.prefix, self.dataset_samples[index])
        chunk_nb, label = self.label_array[index]
        container = av.open(video_file)
        indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames, num_segment = self.num_segment, chunk_nb=chunk_nb)
        video = read_video_pyav(container, indices)
        inputs = self.processor.preprocess(list(video), return_tensors='pt')
        inputs.update(labels=label)
        return inputs

In [ ]:
model = SiglipModel.from_pretrained("google/siglip-so400m-patch14-384") #google/siglip-so400m-patch14-384
model.to(DEVICE)
preprocess = SiglipImageProcessor.from_pretrained("google/siglip-so400m-patch14-384")

In [ ]:
@torch.no_grad()
def extract_feature(data_loader, model):
    model.eval()
    final_result = dict()
    for batch in tqdm(data_loader):
        for k in batch.keys():
            batch[k] = batch[k].to(DEVICE, non_blocking=True)
        B, T, C, H, W = batch['pixel_values'].shape
        batch['pixel_values'] = batch['pixel_values'].flatten(0,1)
        target = batch.pop('labels')
        output =  model.get_image_features(**batch)
        output =  output.reshape(B, T, output.shape[-1])
        for id, feat in zip(target, output):
            id = id.item()
            feat = feat.cpu().numpy()
            if final_result.get(id, None) is None:
                final_result[id] = []
            final_result[id].append(feat)
        break
    for k, v in final_result.items():
        final_result[k] = np.concatenate(final_result[k], axis=0)
    #np.save(file + "_embedding", np.array(list(final_result.values())))
    #np.save(file + "_labels", np.array(list(final_result.keys())))
    return np.array(list(final_result.values())), np.array(list(final_result.keys()))

# FOR DVSC23

In [ ]:
DATASET_NAME = "DVSC23"
DATA_PATH_QUERY=f'annotations/{DATASET_NAME}/queries/test.csv'
DATA_PATH_DATABASE=f'annotations/{DATASET_NAME}/database/test.csv'

QUERY_PREFIX=f'/path/to/prefix_path_of_query'
DATABASE_PREFIX=f'/path/to/prefix_path_of_database'

QUERY_MAP_PATH = f"annotations/{DATASET_NAME}/queries/class_map.json"
DATABASE_MAP_PATH = f"annotations/{DATASET_NAME}/database/class_map.json"

OUTPUT_DIR_QUERY=f'{DATASET_NAME}/queries'
OUTPUT_DIR_DATABASE=f'{DATASET_NAME}/database'
os.makedirs(OUTPUT_DIR_QUERY, exist_ok=True)
os.makedirs(OUTPUT_DIR_DATABASE, exist_ok=True)

NUM_SEGMENTS = 5
BATCH_SIZE = 4

In [ ]:
def normalize(embeddings):
    #embeddings = (embeddings - embeddings.mean(0, keepdims=True)) / (embeddings.std(0, keepdims=True) + 1e-6)
    embeddings /= np.linalg.norm(embeddings, axis=-1, keepdims=True) + 1e-6
    embeddings = (embeddings - embeddings.mean(0, keepdims=True)) / (embeddings.std(0, keepdims=True) + 1e-6)
    return embeddings

def save_features(embeddings, ids, save_path, video_id_map):
    
    def flatten(data, ids):
        T = data.shape[1]
        return data.reshape(-1, data.shape[-1]), np.tile(ids[:, None],(1, T)).flatten()
    embeddings, ids = flatten(embeddings, ids)


    video_ids = [video_id_map[str(id)] for id in ids]  # Can also be str: "Q20000", ...
    sort_ids = np.argsort(video_ids).tolist()
    np.savez(
        save_path,
        video_ids=np.sort(video_ids).tolist(),
        timestamps=[[0, 1]]*len(video_ids),
        features=embeddings[sort_ids].astype(np.float32),
    )

In [ ]:
#extract feature for query in DVSC23
query_dataset = VideoClsDataset(DATA_PATH_QUERY, QUERY_PREFIX, preprocess, NUM_SEGMENTS)
dataloader = DataLoader(query_dataset, num_workers=4, batch_size=BATCH_SIZE)
query_embeddings, query_ids = extract_feature(dataloader, model)
query_embeddings = normalize(query_embeddings)

In [ ]:
with open(QUERY_MAP_PATH, 'r') as file:
    query_map = json.load(file)
save_features(query_embeddings, query_ids, f'{OUTPUT_DIR_QUERY}/descriptors.npz', query_map)

In [ ]:
#extract feature for reference in DVSC23
ref_dataset = VideoClsDataset(DATA_PATH_DATABASE, DATABASE_PREFIX, preprocess, NUM_SEGMENTS)
dataloader = DataLoader(ref_dataset, num_workers=4, batch_size=BATCH_SIZE)
ref_embeddings, ref_ids = extract_feature(dataloader, model)
ref_embeddings = normalize(ref_embeddings)

In [ ]:
with open(DATABASE_MAP_PATH, 'r') as file:
    database_map = json.load(file)
save_features(ref_embeddings, ref_ids, f'{OUTPUT_DIR_DATABASE}/descriptors.npz', database_map)

In [ ]:
%cd DVSC23
!python descriptor_eval.py  \
    --query_features '{OUTPUT_DIR_QUERY}/descriptors.npz' \
    --ref_features '{OUTPUT_DIR_DATABASE}/descriptors.npz' \
    --ground_truth ../annotations/DVSC23/test_ground_truth.csv
%cd ..

# FOR FIVR-5K

In [ ]:
DATA_PATH_QUERY='annotations/FIVR-5K/queries/test.csv'
DATA_PATH_DATABASE='annotations/FIVR-5K/database/test.csv'

QUERY_PREFIX=f'/path/to/prefix_path_of_query'
DATABASE_PREFIX=f'/path/to/prefix_path_of_database'


QUERY_MAP_PATH = "annotations/FIVR-5K/queries/class_map.json"
DATABASE_MAP_PATH = "annotations/FIVR-5K/database/class_map.json"


OUTPUT_DIR='FIVR-5K'
os.makedirs(OUTPUT_DIR, exist_ok=True)

NUM_SEGMENTS = 1
BATCH_SIZE = 4

In [ ]:
def calculate_similarity(test_feature, train_feature):
    test_feature /= np.linalg.norm(test_feature, axis=-1, keepdims=True) + 1e-6
    train_feature /= np.linalg.norm(train_feature, axis=-1, keepdims=True) + 1e-6
    return test_feature @ (train_feature.T)


def save_predictions(logits, save_path, query_map, database_map):
    prediction = dict()
    for query_id, query_prob in enumerate(logits):
        prob_per_query = dict()
        for ref_id, prob in enumerate(query_prob):
            if prob.item() > 0 :
                prob_per_query[str(database_map[str(ref_id)]).split('.')[0]] = prob.item()
        prediction[query_map[str(query_id)].split('.')[0]] = prob_per_query
    with open(save_path, 'w') as f:
        json.dump(prediction, f)

In [ ]:
#extract feature for query in FIVR-5K
query_dataset = VideoClsDataset(DATA_PATH_QUERY, QUERY_PREFIX, preprocess, NUM_SEGMENTS)
dataloader = DataLoader(query_dataset, num_workers=4, batch_size=BATCH_SIZE)
query_embeddings, query_ids = extract_feature(dataloader, model)

In [ ]:
#extract feature for reference in FIVR-5K
ref_dataset = VideoClsDataset(DATA_PATH_DATABASE, DATABASE_PREFIX, preprocess, NUM_SEGMENTS)
dataloader = DataLoader(ref_dataset, num_workers=4, batch_size=BATCH_SIZE)
ref_embeddings, ref_ids = extract_feature(dataloader, model)

In [ ]:
with open(QUERY_MAP_PATH, 'r') as file:
    query_map = json.load(file)

with open(DATABASE_MAP_PATH, 'r') as file:
    database_map = json.load(file)

similarity = calculate_similarity(query_embeddings.mean(1), ref_embeddings.mean(1))
save_predictions(similarity, f"{OUTPUT_DIR}/prediction.json", query_map, database_map)

In [ ]:
%cd FIVR-200K
!python evaluation.py --result_file "../{OUTPUT_DIR}/prediction.json" \
                      --relevant_labels ND,DS \
                      --dataset_ids ../annotations/FIVR-5K/used_videos.txt \
                      --annotations_file ../annotations/FIVR-5K/annotation.json
%cd ..

In [ ]:
%cd FIVR-200K
!python evaluation.py --result_file "../{OUTPUT_DIR}/prediction.json" \
                      --relevant_labels ND,DS,CS \
                      --dataset_ids ../annotations/FIVR-5K/used_videos.txt \
                      --annotations_file ../annotations/FIVR-5K/annotation.json
%cd ..

In [ ]:
%cd FIVR-200K
!python evaluation.py --result_file "../{OUTPUT_DIR}/prediction.json" \
                      --relevant_labels ND,DS,CS,IS \
                      --dataset_ids ../annotations/FIVR-5K/used_videos.txt \
                      --annotations_file ../annotations/FIVR-5K/annotation.json
%cd ..